In [1]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")

Environment Variables are set OK


In [3]:
#Busines Classes

import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str="",systemMessage:str="",query:str=""):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMResult:
    def __init__(self,UUID:str="",result:str="",errorMsg:str="",timeDelta:str=""):
        self.UUID=UUID
        self.result=result
        self.errorMsg=errorMsg
        self.HasError=errorMsg!=""
        self.timeDelta=timeDelta

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
    

In [4]:
#Ollama Processing Code
import datetime
from ollama import Client
client = Client(host='http://localhost:11434')
#client.pull("llama3.1")

#Post to a service bus queue    
def AzurePost_ServiceBus(connection_string: str, queue_name: str, json_payload):
    with ServiceBusClient.from_connection_string(connection_string) as client:
        sender = client.get_queue_sender(queue_name=queue_name)
        message = ServiceBusMessage(json.dumps(json_payload))
        with sender:
            sender.send_messages(message)
            print(f"Queued message: {json_payload['UUID']}")

def ProcessLLMRequest(request:LLMRequest)->LLMResult:
    print("Processing Request "+request.UUID)
    print(request.to_json())
    #ret=client.generate(request.query)
    
    #Start timer
    timerStart=datetime.datetime.now()
    
    #Form suitable message body    
    ret = client.chat(
        model='llama3.1',
        messages=[{'role': 'system', 'content': request.systemMessage},
        {'role': 'user', 'content': request.query}],
        stream=False)
    
    #get result/timing and post back to results queue
    timerEnd=datetime.datetime.now()
    timeDelta=timerEnd-timerStart
    result=LLMResult(UUID=request.UUID,result=ret,timeDelta=str(timeDelta))
    AzurePost_ServiceBus(os.environ.get('Endpoint_Results'),"llm-responses",result.to_json())
    return result
    


In [7]:
#Main Message Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

# Replace with your connection string and queue name
queue_name = "llm-queries"

# Create a Service Bus client
servicebus_client = ServiceBusClient.from_connection_string(conn_str=EndPoint_Queries)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=1, max_wait_time=5)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            # Accept the message to remove it from the queue
            receiver.complete_message(msg)
            
            
            llmRequest=LLMRequest()
            llmRequest.from_json(str(msg))
            print(llmRequest.query)
            result=ProcessLLMRequest(llmRequest)
            print(result.to_json())
            

# Call the function to read from the queue
receive_messages_from_queue()


Receiving messages from the queue...
Received message: {"UUID": "48451cb8-de42-4e82-9487-4371830b0a56", "Model": "llama3.1", "systemMessage": "You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. \nYou have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).\nTry to keep it within medieval fantasy RPG lore and appropriate to this context.\n", "query": "They have the followng traits:\nrace: gnome\ncomplection: dark\nheight: tall\nbuild: muscular\nfeature: round\neye: dull\nhair: short\nclothes: simple\nsophistication: crude\nmannerism: graceful\nvirtue: brave\n"}
They have the followng traits:
race: gnome
complection: dark
height: tall
build: muscular
feature: round
eye: dull
hair: short
clothes: simple
sophistication: crude
mannerism: graceful
virtue: brave

Processing Request 48451cb8-de42-4e82-9487-4371830b0a56
{'UUID': '48451cb8-de42-4e82-9487-437183

In [8]:
#Main loop
while True:
    receive_messages_from_queue()
    #time.sleep(1)

Receiving messages from the queue...
Received message: {"UUID": "6968ff34-f04f-494b-a4d7-f6c707d13e9c", "Model": "llama3.1", "systemMessage": "You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. \nYou have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).\nTry to keep it within medieval fantasy RPG lore and appropriate to this context.\n", "query": "They have the followng traits:\nrace: elf\ncomplection: pale\nheight: tall\nbuild: stocky\nfeature: sharp\neye: dark\nhair: curly\nclothes: gaudy\nsophistication: simple\nmannerism: boisterous\nvirtue: honest\n"}
They have the followng traits:
race: elf
complection: pale
height: tall
build: stocky
feature: sharp
eye: dark
hair: curly
clothes: gaudy
sophistication: simple
mannerism: boisterous
virtue: honest

Processing Request 6968ff34-f04f-494b-a4d7-f6c707d13e9c
{'UUID': '6968ff34-f04f-494b-a4d7-f6c707d1